In [242]:
import subprocess
import zipfile
import os
import numpy as np
import pandas as pd
from sklearn.calibration import LabelEncoder

In [243]:
subprocess.run(['wget', 'http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip', '-O', 'CIC_IDS_2017.zip'], check=True) 

with zipfile.ZipFile("CIC_IDS_2017.zip", 'r') as zip_ref:
    zip_ref.extractall("./data")
    
df_list = []
for file in os.listdir('./data/MachineLearningCVE/'):
  df_aux = pd.read_csv(f'./data/MachineLearningCVE/{file}')
  df_list.append(df_aux)

--2025-08-12 00:02:48--  http://205.174.165.80/CICDataset/CIC-IDS-2017/Dataset/CIC-IDS-2017/CSVs/MachineLearningCSV.zip
Connecting to 205.174.165.80:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 235102953 (224M) [application/zip]
Saving to: ‘CIC_IDS_2017.zip’

     0K .......... .......... .......... .......... ..........  0%  801K 4m47s
    50K .......... .......... .......... .......... ..........  0%  871K 4m35s
   100K .......... .......... .......... .

......... ..........  0% 1.56M 3m51s
   150K .......... .......... .......... .......... ..........  0% 1.69M 3m27s
   200K .......... .......... .......... .......... ..........  0% 1.71M 3m11s
   250K .......... .......... .......... .......... ..........  0% 1.71M 3m1s
   300K .......... .......... .......... .......... ..........  0% 1.74M 2m54s
   350K .......... .......... .......... .......... ..........  0% 4.47M 2m38s
   400K .......... .......... .......... .......... ..........  0% 2.58M 2m30s
   450K .......... .......... .......... .......... ..........  0% 1.76M 2m28s
   500K .......... .......... .......... .......... ..........  0% 22.1M 2m15s
   550K .......... .......... .......... .......... ..........  0% 1.82M 2m14s
   600K .......... .......... .......... .......... ..........  0% 24.9M 2m5s
   650K .......... .......... .......... .......... ..........  0% 1.75M 2m5s
   700K .......... .......... .......... .......... ..........  0% 40.4M 1m57s
   750K ..........

In [244]:
df = pd.concat(df_list, ignore_index=True)

In [245]:
df.columns = df.columns.str.strip()

In [246]:
features = df.dtypes[df.dtypes != 'object'].index
df[features] = df[features].apply(
    lambda x: (x - x.mean()) / (x.std()))

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/pandas/core/nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [247]:
df = df.fillna(0)

labelencoder = LabelEncoder()
df.iloc[:, -1] = labelencoder.fit_transform(df.iloc[:, -1])

In [248]:
# After fitting the labelencoder
original_labels = labelencoder.classes_

# Create a mapping from original label to encoded value
label_mapping = {label: idx for idx, label in enumerate(original_labels)}

print("Label Mapping:")
for label, code in label_mapping.items():
    print(f"{label}: {code}")


Label Mapping:
BENIGN: 0
Bot: 1
DDoS: 2
DoS GoldenEye: 3
DoS Hulk: 4
DoS Slowhttptest: 5
DoS slowloris: 6
FTP-Patator: 7
Heartbleed: 8
Infiltration: 9
PortScan: 10
SSH-Patator: 11
Web Attack � Brute Force: 12
Web Attack � Sql Injection: 13
Web Attack � XSS: 14


In [249]:
df.Label.value_counts()

Label
0     2273097
4      231073
10     158930
2      128027
3       10293
7        7938
11       5897
6        5796
5        5499
1        1966
12       1507
14        652
9          36
13         21
8          11
Name: count, dtype: int64

In [250]:
df_minor = df[(df['Label'] == 9) | (df['Label'] == 8) | (df['Label'] == 13) | (df['Label'] == 14) | (df['Label'] == 12) | (df['Label'] == 1) ]
df_major = df.drop(df_minor.index)

In [251]:
X = df_major.drop(['Label'],axis=1)
y = df_major.iloc[:, -1].values.reshape(-1,1)
y=np.ravel(y)

In [252]:
from sklearn.cluster import MiniBatchKMeans
# KMeans clustering
kmeans = MiniBatchKMeans(n_clusters=1000, random_state=0).fit(X)
df_major['klabel'] = kmeans.labels_

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


In [253]:
cols = list(df_major)
cols.insert(78, cols.pop(cols.index('Label')))
df_major = df_major.loc[:, cols]

In [254]:
def benignSampling(group):
    return group.sample(frac=0.02)

In [255]:
def typicalSampling(group):
    return group.sample(frac=0.0045)

In [256]:
df_label_0 = df_major[df_major['Label'] == 0]
df_other_labels = df_major[df_major['Label'] != 0]


result_0 = df_label_0.groupby(
    'klabel', group_keys=False
).apply(typicalSampling)

result_1 = df_other_labels.groupby(
    'klabel', group_keys=False
).apply(benignSampling)


In [257]:
result = pd.concat([result_0, result_1, df_minor], ignore_index=True)
result = result.drop(['klabel'], axis=1)

In [258]:
# Save in the same directory
result.to_csv('CICIDS2017_sample_km.csv', index=False)
print("Final sampled dataset saved to CICIDS2018_sample_km.csv")

Final sampled dataset saved to CICIDS2018_sample_km.csv


In [259]:
df = pd.read_csv('CICIDS2017_sample_km.csv')

In [260]:
df.Label.value_counts()

Label
0     10226
4      4619
10     3178
2      2564
1      1966
12     1507
14      652
3       208
7       155
6       118
11      116
5       113
9        36
13       21
8        11
Name: count, dtype: int64